# Sentinel 2 CH4 Multi Band Multi Pass Mapper

## Overview 
Varon et al. (2021) showed that methane plumes from point sources could be imaged by differencing Sentinel-2’s SWIR-1 and SWIR-2 bands. The tool runs an analysis using a  multi-band-multi-pass retrieval method: 

First it calculates a multi-band-single-pass calculation for both active emission and no emission dates, resulting in two datasets which are then used together for a multi-band-multi-pass method. 
The multi-band-single-pass equation is as follows: 


<div align="center"><b>MBSP = B11 - cB12</b></div>

Where:
- B12 is the Sentinel-2 SWIR-2 band.
- B11 is the Sentinel-2 SWIR-1 band. 
- c is calculated by least-squares fitting B12 to B11 across the scene.  

Once active emission and no emission scenes have been calculated, the following equation is used to calculate the multi-band-multi-pass raster. 

<div align="center"><b>MBMP = ActiveMBSP − NoMBSP</b></div>

Where:
- ActiveMBSP is the multiband single pass for the active emission scene
- NoMBSP is the multiband single pass for the no emission scene.  

The active emission scene and no emission scene are considered in this analysis to be one satelite pass apart.

The section below imports the packages needed to run the script.

In [ ]:
import os
import folium
import pandas
import matplotlib.pyplot as plt
import openeo
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import requests
from folium import Map, LayerControl, LatLngPopup, GeoJson
from folium.raster_layers import ImageOverlay
from IPython.display import display
from skimage import exposure
import geopandas as gpd
import pandas as pd
from PIL import Image
from scipy.ndimage import label
from scipy.spatial import ConvexHull


## Connect to OpenEO

The code below establishes a connection with the Copernicus openEO platform which provides a wide variety of earth observation datasets

- If this does not read as 'Authorised successfully' or 'Authenticated using refresh token', then please ensure that you have completed the setup steps as outlined in section 2.6 of the user guide. 

- If you have followed the steps in section 2.6 correctly and the problem persists, please look at https://dataspace.copernicus.eu/news for any information about service interruptions. 

- If there is no news of service problems you can raise a ticket here: https://helpcenter.dataspace.copernicus.eu/hc/en-gb/requests/new

In [ ]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

## Load Study Area. 

This loads the boudings for the oil and gas fields in Algeria. Hassi Messaoud is site 86

In [ ]:
studysite_csv = pandas.read_csv(r'C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Data\Algerian_Oil_and_Gas_Fields.csv')
pandas.set_option('display.max_rows', None)
print(studysite_csv.to_string(index=False))

# Available dates for the analysis. 

Sentinel 2 provides data aproximately once every 3 days, so not every date you can enter into this tool is valid. The code below will tell you what dates are available to use for the landfill of your choice. 

The two parameters you need to modify before running the code are: 
- landfill_id = 86 (change this to your chosen study site) 
- temporal_extent = ["2023-01-31", "2023-03-12"] (change this to your chosen date range using "YYYY-MM-DD" format)

Once you have done this run the code and the available dates should appear below in a matter of seconds. 

In [ ]:
def get_spatial_extent(site_id):
    site = studysite_csv[studysite_csv['id'] == site_id].iloc[0]
    return {
        "west": site['west'],
        "south": site['south'],
        "east": site['east'],
        "north": site['north']
    }

def fetch_available_dates(site_id, temporal_extent):
    spatial_extent = get_spatial_extent(site_id)
    catalog_url = f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?box={spatial_extent['west']}%2C{spatial_extent['south']}%2C{spatial_extent['east']}%2C{spatial_extent['north']}&sortParam=startDate&sortOrder=ascending&page=1&maxRecords=1000&status=ONLINE&dataset=ESA-DATASET&productType=L2A&startDate={temporal_extent[0]}T00%3A00%3A00Z&completionDate={temporal_extent[1]}T00%3A00%3A00Z&cloudCover=%5B0%2C{cloud_cover}%5D"
    response = requests.get(catalog_url)
    response.raise_for_status()
    catalog = response.json()
    dates = [date.split('T')[0] for date in map(lambda x: x['properties']['startDate'], catalog['features'])]
    return dates

# Please enter your perameters here.
site_id = 86 # Specify the oil and gas field ID
temporal_extent = ["2021-01-01", "2021-01-31"]  # Specify the the date range you want to check for available data.
cloud_cover = 5

available_dates = fetch_available_dates(site_id, temporal_extent)
print("Available dates:", available_dates)

## Choosing the Active Emission Date

As mentioned in the overview, an active emission date must be chosen from one of the available datasets. 

Like before, the two parameters you need to modify before running the code are:

- site_id = 86 (change this to your chosen study site)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.) 

Please note that the temporal extent dates <u>MUST BE IDENTICAL</u> because we are only choosing a single date.

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.

In [ ]:
def active_emission(site_id, temporal_extent):
    site = studysite_csv[studysite_csv['id'] == site_id].iloc[0]

    active_emission = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": site['west'],
            "south": site['south'],
            "east": site['east'],
            "north": site['north']
        },
        bands=["B11", "B12"],
    )
    active_emission.download("Sentinel-2_active_emissionMBMP.Tiff")

# Enter perameters for the active emission day
site_id = 86  # Specify the oil and gas field ID
temporal_extent = ["2021-01-11", "2021-01-11"]

active_emission(site_id, temporal_extent)

## Choosing the No Emission Date

Next we choose the no emission date using the same process. 

The two parameters you need to modify before running the code are:

- site_id = 86 (change this to your chosen site)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.) 

The temporal extent dates <u>MUST BE IDENTICAL</u>

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.


In [ ]:
def no_emission(site_id, temporal_extent):
    site = studysite_csv[studysite_csv['id'] == site_id].iloc[0]

    no_emission = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": site['west'],
            "south": site['south'],
            "east": site['east'],
            "north": site['north']
        },
        bands=["B11", "B12"],
    )
    no_emission.download("Sentinel-2_no_emissionMBMP.Tiff")

# Enter perameters for the active emission day
site_id = 86  # Specify the oil and gas field ID
temporal_extent = ["2021-01-08", "2021-01-08"]

no_emission(site_id, temporal_extent)

## Choosing a Background Satelite Image

This section helps with locating the source of the emission at the landfill by displaying a true colour satelite image of the landfill that the data will be superimposed over. I recommend choosing the same date as your active emission. 

Once again, the two parameters you need to modify before running the code are:

- site_id = 86 (change this to your chosen site)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.)

The temporal extent dates <u>MUST BE IDENTICAL</u>

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.

In [ ]:
def truecolour_image(site_id, temporal_extent):
    site = studysite_csv[studysite_csv['id'] == site_id].iloc[0]

    truecolour_image = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": site['west'],
            "south": site['south'],
            "east": site['east'],
            "north": site['north']
        },
        bands=["B02", "B03", "B04"],
    )
    truecolour_image.download("Sentinel-2_truecolourMBMP.Tiff")

# Enter parameters for the no emission day
site_id = 86  # Specify the oil and gas field ID
temporal_extent = ["2021-01-11", "2021-01-11"]

truecolour_image(site_id, temporal_extent)

## Running the Analysis
The code below runs the analysis. Provided all the variables above have been run correctly, this next section should take moments to complete. 

In [ ]:
# Define file paths
Active_Multiband = "Sentinel-2_active_emissionMBMP.Tiff"
No_Multiband = "Sentinel-2_no_emissionMBMP.Tiff"

# Define a function for least squares fitting
def least_squares_fit(x, y):
    # Remove NaNs (if any) for valid calculations
    mask = ~np.isnan(x) & ~np.isnan(y)
    x_valid = x[mask]
    y_valid = y[mask]
    
    # Calculate least squares fit parameters
    A = np.vstack([x_valid, np.ones_like(x_valid)]).T
    m, c = np.linalg.lstsq(A, y_valid, rcond=None)[0]
    return m, c

# Open datasets and perform least squares fitting
with rasterio.open(Active_Multiband) as Active_img, rasterio.open(No_Multiband) as No_img:
    Active_B11 = Active_img.read(1)
    Active_B12 = Active_img.read(2)
    No_B11 = No_img.read(1)
    No_B12 = No_img.read(2)
    
    # Perform least squares fitting for Active_B11 vs Active_B12
    m_active, c_active = least_squares_fit(Active_B11.flatten(), Active_B12.flatten())
    Corrected_Active_B12 = m_active * Active_B12 + c_active
    
    # Perform least squares fitting for No_B11 vs No_B12
    m_no, c_no = least_squares_fit(No_B11.flatten(), No_B12.flatten())
    Corrected_No_B12 = m_no * No_B12 + c_no
    
    # Calculate the fractional change
    SWIR_diff = (Active_B11 - Corrected_Active_B12) - (No_B11 - Corrected_No_B12)



## Viewing the data. 

This section of code can be run to produce the map. Three peramaters can to be adjusted. 

- site_id = 86 (change this to your chosen site)
- brightness_factor = 0.05 (occasionally the true colour satelite image can be too bright or too dark. You can change this number to fix it)

In [ ]:
# Function to get bounds from the Oil and Gas Field bounding file
def get_bounds(site_id, csv_path):
    # Load CSV data
    df = pd.read_csv(csv_path)

    # Take the data from a particular record
    site = df[df['id'] == site_id]
    if site.empty:
        raise ValueError(f"Site ID {site_id} not found in the CSV file.")

    # bounding box extents
    site = site.iloc[0]
    return [[site['south'], site['west']], [site['north'], site['east']]]

# Specify the site ID and input paths
site_id = 86  # Specify the oil and gas field ID
csv_path = r'C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Data\Algerian_Oil_and_Gas_Fields.csv'
bounds = get_bounds(site_id, csv_path)

# Calculate the center of the map
center_lat = (bounds[0][0] + bounds[1][0]) / 2
center_lon = (bounds[0][1] + bounds[1][1]) / 2

# Load the true color image
truecolour_sat = 'Sentinel-2_truecolourMBMP.Tiff'
img = rasterio.open(truecolour_sat)
blue, green, red = img.read(1), img.read(2), img.read(3)

# Adjust brightness dynamically
brightness_factor = 0.03  # Brightness adjustment factor
blue = np.clip(blue * brightness_factor, 0, 255)
green = np.clip(green * brightness_factor, 0, 255)
red = np.clip(red * brightness_factor, 0, 255)

# Stack bands to create RGB image
rgb = np.dstack((red, green, blue))
rgb = rgb / rgb.max()  # Normalize values between 0 and 1

# Apply logarithmic transformation to enhance contrast
rgb = np.log1p(rgb)
rgb = rgb / rgb.max()  # Normalize again after transformation

# Create a Folium map with the dynamic center
m = Map(location=[center_lat, center_lon], zoom_start=10, control_scale=True)

# Add true color image overlay with dynamic bounds
truecolour_overlay = ImageOverlay(
    image=rgb,
    bounds=bounds,
    opacity=1,
    interactive=True,
    cross_origin=False,
    zindex=1,
)
truecolour_overlay.add_to(m)

# Assuming SWIR_diff has been calculated earlier in your workflow
# Calculate mean and standard deviation of SWIR_diff
mean, std = np.nanmean(SWIR_diff), np.nanstd(SWIR_diff)

# Perform standard deviation stretch
std_factor = 2  # Number of standard deviations for stretching
lower_bound, upper_bound = mean - std_factor * std, mean + std_factor * std

# Normalize SWIR_diff using standard deviation stretch
normalized_SWIR_diff = (SWIR_diff - lower_bound) / (upper_bound - lower_bound)
normalized_SWIR_diff = np.clip(normalized_SWIR_diff, 0, 1)  # Clip values to [0, 1]

# Apply colormap to normalized SWIR_diff
cmap = plt.get_cmap('plasma')
SWIR_colored = cmap(normalized_SWIR_diff)

# Add SWIR_diff overlay with dynamic bounds
SWIR_overlay = ImageOverlay(
    image=SWIR_colored,
    bounds=bounds,
    opacity=1,
    interactive=True,
    cross_origin=False,
    zindex=2,
)
SWIR_overlay.add_to(m)

# Load GeoJSON file for known point sources
vector_point_path = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Data\known_point_sources.geojson"
gdf = gpd.read_file(vector_point_path)

# Add GeoJSON overlay to the map
GeoJson(gdf.to_json()).add_to(m)

# Add a layer control to toggle map layers
LayerControl().add_to(m)

# Add a click event to display latitude and longitude on the map
m.add_child(LatLngPopup())

# Display the map
display(m)

# File saving output name
output_file = "SWIR_diff_4326.tiff"

# Reproject the raster to EPSG:4326
source_crs = "EPSG:32632"  # Current CRS
target_crs = "EPSG:4326"  # Target CRS

# Calculate transform and metadata for the target CRS
transform, width, height = calculate_default_transform(
    source_crs, target_crs, img.width, img.height, *img.bounds
)

# Update metadata for the target CRS
meta = img.meta.copy()
meta.update({
    "crs": target_crs,
    "transform": transform,
    "width": width,
    "height": height,
    "count": 1,  # Single band for SWIR_diff
    "dtype": SWIR_diff.dtype
})

# Save reprojected raster directly
with rasterio.open(output_file, "w", **meta) as dest:
    reproject(
        source=SWIR_diff,
        destination=rasterio.band(dest, 1),
        src_transform=img.transform,
        src_crs=source_crs,
        dst_transform=transform,
        dst_crs=target_crs,
        resampling=Resampling.nearest
    )


## Plume tagging

Maually input plume source coordinates here in the format (latitude, longitude), for example:  
    (31.6887, 5.8102),  # Plume 1 (latitude, longitude)  
    (31.7910, 5.8263),  # Plume 2 (latitude, longitude)  
etc...

Plumes that are segmented will need to have each of their segmets taged to be included in the analysis. Additional lines for more plumes can be added as needed.

In [ ]:
plume_coords = [
    (31.6887, 5.8102),  # Plume 1 (latitude, longitude)
    (31.7910, 5.8263),  # Plume 2 (latitude, longitude)
    (31.7978, 5.8341),  # Plume 4 (latitude, longitude)
    (31.9101, 6.0135),  # Plume 3 (latitude, longitude)
    (31.6389, 6.0025),  # Plume 4 (latitude, longitude)
]

## Loading and configuring map.

This section loads the SWIR data and loads the colourmap in preparation for the analysis. It also provides the average/mean value of the dataset, allowing us to see how much a plume rises above background levels. 

In [ ]:
# Load TIFF file
tiff_file_path = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\SWIR_diff_4326.tiff"
with rasterio.open(tiff_file_path) as tiff_file:
    raster_data = tiff_file.read(1)  # Read the first band
    nodata_value = tiff_file.nodata if tiff_file.nodata is not None else -9999
    bounds = tiff_file.bounds
    transform = tiff_file.transform

# Mask nodata values
masked_data = np.ma.masked_equal(raster_data, nodata_value)
mean, std = np.nanmean(masked_data), np.nanstd(masked_data)
std_factor = 2
lower_bound, upper_bound = mean - std_factor * std, mean + std_factor * std
normalized_data = (masked_data - lower_bound) / (upper_bound - lower_bound)
normalized_data = np.clip(normalized_data, 0, 1)  # Clip to [0, 1]

# Apply colormap
cmap = plt.get_cmap('plasma')
rgb_data = (cmap(normalized_data)[:, :, :3] * 255).astype(np.uint8)

# Initialize the map
center_lat = (bounds.top + bounds.bottom) / 2
center_lon = (bounds.left + bounds.right) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=11, control_scale=True)

# Add SWIR_diff overlay
image_bounds = [[bounds.bottom, bounds.left], [bounds.top, bounds.right]]
swir_overlay = ImageOverlay(
    image=rgb_data,
    bounds=image_bounds,
    opacity=1,
    interactive=True,
    cross_origin=False,
    zindex=1,
)
swir_overlay.add_to(m)

# Mark plume locations
for i, (lat, lon) in enumerate(plume_coords):
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=1,
        popup=f"Plume {i + 1}",
    ).add_to(m)

# Calculate the mean value of the masked data
dataset_mean_value = masked_data.mean()

# Print the mean value
print(f"Mean value of the dataset: {dataset_mean_value}")

## Viewing the data. 

Now everything is loaded, we can run the analysis. 

In [ ]:
# Analyze plumes
def analyze_plume(masked_data, plume_coords, transform):
    plume_results = []
    labeled_array, _ = label(masked_data > np.percentile(masked_data.compressed(), 65))  # Identify plumes
    for i, (lat, lon) in enumerate(plume_coords):
        try:
            row, col = rasterio.transform.rowcol(transform, lon, lat)
            row, col = int(row), int(col)
            plume_label = labeled_array[row, col]
            if plume_label == 0:
                plume_results.append({
                    "Plume": i + 1,
                    "Location (lat, lon)": (lat, lon),
                    "Status": "No plume detected",
                })
            else:
                plume_region = labeled_array == plume_label
                plume_values = masked_data[plume_region]
                plume_pixels = np.column_stack(np.where(plume_region))

                # Convert pixel coordinates to lat/lon
                plume_longitudes, plume_latitudes = rasterio.transform.xy(
                    transform, plume_pixels[:, 0], plume_pixels[:, 1]
                )
                points = np.array(list(zip(plume_latitudes, plume_longitudes)))
                hull = ConvexHull(points)
                polygon_coordinates = [(points[vertex, 0], points[vertex, 1]) for vertex in hull.vertices]

                # Add polygon to the map
                folium.Polygon(
                    locations=polygon_coordinates,
                    color="blue",
                    weight=1,
                    fill=True,
                    fill_color="blue",
                    fill_opacity=0.4,
                    popup=f"Plume {i + 1} region",
                ).add_to(m)

                plume_results.append({
                    "Plume": i + 1,
                    "Location (lat, lon)": (lat, lon),
                    "Max Value": plume_values.max(),
                    "Mean Value": plume_values.mean(),
                    "Size (pixels)": plume_region.sum(),
                })
        except Exception as e:
            plume_results.append({
                "Plume": i + 1,
                "Location (lat, lon)": (lat, lon),
                "Status": f"Error: {e}",
            })
    return plume_results

# Perform analysis
plume_results = analyze_plume(masked_data, plume_coords, transform)

# Convert results to DataFrame
plume_df = pd.DataFrame([{
    "Plume": result["Plume"],
    "Location (lat, lon)": result["Location (lat, lon)"],
    "Max Value": result.get("Max Value", None),
    "Mean Value": result.get("Mean Value", None),
    "Size (pixels)": result.get("Size (pixels)", None),
    "Status": result.get("Status", "Plume detected"),
} for result in plume_results])

# Display DataFrame
print(plume_df)

# Add a layer control and click event
LayerControl().add_to(m)
m.add_child(LatLngPopup())

# Display the map
m